Imports

In [96]:
from gurobipy import *
import numpy as np

from db_manager import *
from load_data import *
from printing import *

In [97]:
# clases
from entities import *
from variables import *

# Cargar datos

## (1) Instancia

In [98]:

instance = "../instances/instance_2026sem1.json"
dias, turnos, horarios, bloques_horario, grupos, materias, profesores, superposicion, superposicion_electivas, num_salones = read_json_instance(instance)

In [99]:

dias_ids = [d.id for d in dias]
DIAS = dias

horarios_ids = [h.id for h in horarios]
HORARIOS = horarios

bloques_horario_ids = [b for b in bloques_horario]
BLOQUES_HORARIO = bloques_horario

profesores_ids = [p.id for p in profesores]
PROFESORES = profesores

grupos_ids = [g.id for g in grupos]
GRUPOS = grupos

anios = list(set([g.anio for g in grupos if g.anio != None]))  # anios
ANIOS = anios

materias_ids = [m.id for m in materias]
MATERIAS = materias

SUPERPOSICION = superposicion
SUPERPOSICION_ELECTIVAS = superposicion_electivas



In [100]:
# # print(*[str(g) for g in grupos], sep="\n")
# print(*[m.profesores for m in materias], sep="\n")

In [101]:
print_prioridades(dias, horarios, profesores)


 Profesor: 3 3  |> last update: None
	1	2	3	4	5
·····················································
0_start-0_end	-	-	3	3	-
1_start-1_end	-	-	1	1	-
2_start-2_end	-	-	1	1	-
3_start-3_end	-	-	-	-	-
4_start-4_end	-	-	-	-	-
5_start-5_end	-	-	-	-	-
6_start-6_end	-	-	-	-	-
7_start-7_end	-	-	-	-	-
8_start-8_end	-	-	-	-	-
9_start-9_end	2	2	-	-	2
10_start-10_end	1	1	-	-	1
11_start-11_end	1	1	-	-	1
12_start-12_end	3	3	-	-	3
13_start-13_end	-	-	-	-	-
14_start-14_end	-	-	-	-	-
15_start-15_end	-	-	-	-	-

 Profesor: 4 4  |> last update: None
	1	2	3	4	5
·····················································
0_start-0_end	-	-	-	-	-
1_start-1_end	1	1	-	-	1
2_start-2_end	1	1	-	-	1
3_start-3_end	1	1	-	-	1
4_start-4_end	1	1	-	-	1
5_start-5_end	1	1	-	-	1
6_start-6_end	-	-	-	-	-
7_start-7_end	-	1	-	-	1
8_start-8_end	-	1	-	-	1
9_start-9_end	-	1	-	-	1
10_start-10_end	-	1	-	-	1
11_start-11_end	-	-	-	-	-
12_start-12_end	-	-	-	-	-
13_start-13_end	-	-	-	-	-
14_start-14_end	-	-	-	-	-
15_start-15_end	-	-	-	-	-

 

In [102]:
print("Grupos:", len(grupos))
# print([str(i) for i in grupos])
print("Materias:", len(materias))
# print([str(i) for i in materias])
print("Profesores:", len(profesores))
# print([str(i) for i in profesores])

Grupos: 29
Materias: 165
Profesores: 117


# Formulacion

## (2) Variables


$$ u_{mb} \in \{0,1\} $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} \in \{0,1\} $$

    m: materia
    d: dia

$$ w_{mp} \in \{0,1\} $$

    m: materia
    p: profesor


$$ x_{gb} \in \{0,1\} $$

    g: grupo
    b: bloque horario = [dh]: dia y hora

$$ y_{pb} \in \{0,1\} $$

    p: profesor
    b: bloque horario = [dh]: dia y hora

$$ z_{pd} \in \{0,1\} $$

    p: profesor
    d: dia

In [103]:
print(len(materias))
print(len(bloques_horario))

165
80


## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion (redundante con la definicion de la variable x)
$$ \sum_{b} \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$


In [104]:
"""
    This code defines constraints to ensure that no two subjects (materias) overlap
    in the same time block (bloque horario).
    Constraints:
        1. For each time block (b) in bloques_horario_ids, the sum of the product of the decision variables 
        (u_dict[m1, b].variable and u_dict[m2, b].variable) and the overlap value (superposicion[(m1, m2)].value)
        for all pairs of subjects (m1, m2) must be equal to 0.
"""

def constr_superposicion(model, u_dict):

    # model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
    #                         for m1 in materias_ids for m2 in materias_ids)
    #             == 0 for b in bloques_horario_ids)
    
    for b in bloques_horario_ids:
        for m1 in materias_ids:
            for m2 in materias_ids:
                if m1 != m2 and superposicion[(m1, m2)].value == 1:
                    model.addConstr(u_dict[m1, b].variable + u_dict[m2, b].variable <= 1)
        


#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [105]:
def constr_carga_horaria(model, u_dict):
    for m in materias:
        model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) particion de horas por materia

##### (3.1.3.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [106]:
def constr_dias_materia(model, v_dict):
    for m in materias:
        model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.3.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [107]:
def constr_max_min_horas(model, u_dict, v_dict):

    for m in materias:

        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
        
        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)
    

##### (3.1.3.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turnos
    para todo m



In [108]:
def constr_turnos_materia(model, u_dict):

    for m in materias:
        no_bloques_materia_ids = [b for b in bloques_horario if b not in bloques_horario_materia(m, bloques_horario)]
        model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.4) horas consecutivas dentro de un dia

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [109]:
def constr_horas_consecutivas(model, u_dict, v_dict):

    model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

Alternativa:
$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} \leq 1 $$
    para todo m, d

#### (3.1.5) evitar dias consecutivos para una misma materia

$$ \sum_m \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$


In [110]:
mats_dias_consecutivos = [mat for mat in materias if mat.dias_consecutivos]

def constr_dias_consecutivos(model, v_dict):

    model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                 for m in [mat.id for mat in materias if not mat in mats_dias_consecutivos])


### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ \sum_m w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [111]:
def constr_no_disponible_profesor(model, u_dict, w_dict):
    for m in materias:
        for p in m.profesores:
            for b in p.no_disponible:
                model.addConstr(u_dict[m.id, b.id].variable * w_dict[m.id, p.id].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

REDUNDANTE CON DEFINICION DE y_pb

In [112]:
p_grupos_simultaneos = [p.nombre for p in profesores if p.cursos_simultaneos]

def constr_unica_materia_profesor(model, u_dict, w_dict):

    for p in profesores:
        mats_p = [m.id for m in materias_profesor(p, materias)]
        if p.nombre not in p_grupos_simultaneos:
            model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in mats_p)
                        <= 1 for b in bloques_horario_ids)
        else: # caso particular
            model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in mats_p)
                     <= 2 for b in bloques_horario_ids)
        

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [113]:
def constr_limitar_profesores_materia(model, w_dict):
    
    for m in materias:
        for p in profesores:
            if p not in m.profesores:
                model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) cantidad de profesores para una misma materia

$$ \sum_p {w_{mp}} = P_m $$

    para todo m

In [114]:
def constr_cantidad_profesores(model, w_dict):
    for m in materias:
        model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == m.cantidad_profesores)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} \leq K_p $$

    para todo p



In [115]:
# separar por nombre de materia
def constr_grupos_max_profesor(model, w_dict):
    for p in profesores:
        for l in p.lista_materias:
            mats = search_materias_by_nombre(materias, l["nombre_materia"]) # subject_id
            if len(mats) > 0:
                model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

#### (3.2.5) definicion de variable "y" (asignacion horario-profesor)

$$ y_{pb} = \sum_m{u_{mb}·w_{mp}} $$

$$ y_{pb} = \text{OR}_m \{u_{mb}·w_{mp}\} $$

    para todo p, b

In [116]:
def constr_definir_y(model, y_dict, u_dict, w_dict):

    for p in profesores:
        uw_vars = {} # variable auxiliar = u_mb·w_mp
        mats_p = materias_profesor(p, materias)
        for m in mats_p:
            # for b_id in bloques_horario_materia(m, bloques_horario):
            for b_id in bloques_horario_ids:
                uw_vars[m.id,b_id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+str(b_id)+str(p)+"_uw")
                model.addConstr(uw_vars[m.id,b_id] == u_dict[m.id,b_id].variable * w_dict[m.id,p.id].variable)
        
        mats_p_ids = [m.id for m in mats_p]
        model.addConstrs(y_dict[p.id,b_id].variable ==
                     gp.or_(uw_vars[m_id,b_id] for m_id in mats_p_ids)
                     for b_id in bloques_horario_ids)
    
    # model.addConstrs(y_dict[p, b].variable ==
    #              gp.quicksum(u_dict[m,b].variable * w_dict[m,p].variable for m in materias_ids)
    #              for b in bloques_horario_ids for p in profesores_ids)

#### (3.2.6) definicion de variable "z" (para dias con clase por profesor)

$$ z_{pd} = OR_h{y_{p(dh)}} $$

    para todo p, d

In [117]:
def constr_definir_z(model, z_dict, y_dict):
    
    model.addConstrs(z_dict[p,d].variable ==
                gp.or_(y_dict[p,(d,h)].variable for h in horarios_ids)
                for p in profesores_ids for d in dias_ids)

### (3.3) Restricciones externas

#### (3.3.1) cantidad de salones

$$ \sum_{m} {u_{mb}} \leq K $$

    para todo b

In [118]:
def constr_cantidad_salones(model, u_dict, num_salones):
    model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= num_salones for b in bloques_horario_ids)

#### (3.3.2) restricciones ad hoc

#### (3.3.3) practico despues del teorico

$$ \max_b\{(-b)*u_{mb}\} > \max_b\{(-b)*u_{m'b}\} $$

    para todo par (teorico, practico) =  (m, m')


In [119]:
def constr_teo_prac(model, u_dict):

    """
    Variables:
        teo (list): Array containing IDs of theoretical subjects.
        prac (list): Array containing IDs of practical subjects.
        vars1erHora (dict): Dictionary mapping subject IDs to Gurobi integer variables representing the first hour.
        scaled_u_dict (dict): Dictionary mapping tuples of subject IDs and block IDs to scaled Gurobi integer variables.

    Functions:
        search_materias_by_nombre(materias, nombre): Searches for subjects by name in the given list of subjects.

    Workflow:
    1. Populate `teo` and `vars1erHora` with IDs and variables for theoretical subjects.
    2. Populate `prac` and `vars1erHora` with IDs and variables for practical subjects.
    3. Create scaled variables `scaled_u_dict` for each subject and block combination.
    4. Add constraints to the model to ensure that `vars1erHora` variables are the maximum of the scaled variables.
    5. Add constraints to ensure that the first hour of theoretical subjects is greater than or equal to the first hour of practical subjects if the lengths of `teo` and `prac` are equal.


    """
    teo = []    # array con IDS de materias de teorico
    prac = []    # array con IDS de materias de practico
    vars1erHora = {}
    scaled_u_dict = {}

    for t in [m for m in materias if m.teo_prac == "teo"]:
        teo.append(t.id)
        vars1erHora[t.id] = model.addVar(vtype=GRB.INTEGER, name=str(t)+"_1er_hora")

    for p in [m for m in materias if m.teo_prac == "prac"]:
        prac.append(p.id)
        vars1erHora[p.id] = model.addVar(vtype=GRB.INTEGER, name=str(p)+"_1er_hora")

    L = len(bloques_horario_ids)

    for m in teo + prac:
        for b in range(0, L):
            scaled_u_dict[m, bloques_horario_ids[b]] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"scaled_u")
            model.addConstr(scaled_u_dict[m, bloques_horario_ids[b]] == (L - b) * u_dict[m, bloques_horario_ids[b]].variable)

    for m_id in vars1erHora:
        model.addConstr(vars1erHora[m_id] ==
                        # gp.max_( (L - b) * u_dict[m_id, bloques_horario_ids[b]].variable
                        gp.max_( scaled_u_dict[m_id, bloques_horario_ids[b]]
                                for b in range(0, L)) )


    if len(teo) == len(prac):
        model.addConstrs(vars1erHora[teo[m]] >= vars1erHora[prac[m]] for m in range(0, len(teo)))
    else:
        print("Error: no coinciden cantidad de teoricos con practicos")


#### (3.3.4) Limitar horas excepcionales

$$ \sum_{m} \sum_{b \in \mathcal{E}_m} {u_{mb}} \leq E$$

In [120]:

def constr_horarios_excepcionales(model, u_dict, E_teo, E_prac):

    suma_horas_prac = gp.LinExpr()
    suma_horas_teo = gp.LinExpr()
    
    for m in materias:
        for b in bloques_horario.values():
            excep = b.horario.turnos_excepcional
            if len(intersection(excep, m.turnos())) > 0:
                if m.teo_prac == "prac":
                    suma_horas_prac += u_dict[m.id, b.id].variable
                else:
                    suma_horas_teo += u_dict[m.id, b.id].variable
    
    model.addConstr(suma_horas_prac <= E_prac, "horas_excepcionales_prac")
    model.addConstr(suma_horas_teo <= E_teo, "horas_excepcionales_teo")
    

### (3.4) Grupos

#### (3.4.1) definicion de variable "x"

$$ x_{gb} = OR_{m \in M_g} {u_{mb}} $$



In [121]:
def constr_definir_x(model, x_dict, u_dict):

    for g in grupos:
        # gr_mats_ids = [m.id for m in materias_grupo(g, materias)]
        gr_mats_ids = [m.id for m in materias_grupo(g, [m for m in materias if not m.electiva])] # filtrar materias electivas
        model.addConstrs(x_dict[g.id, b].variable == gp.or_(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)



#### (3.4.2) evitar horas puente por grupo

$$ \sum_h {x_{gdh}} - \sum_h {x_{gd(h)} · x_{gd(h+1)}} \leq 1 $$
    para todo g, d

In [122]:
def constr_horas_puente_grupos(model, x_dict):

    for g in grupos:
        model.addConstrs(gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
                    - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
                    <= 1 for d in dias_ids)

## (4) Funcion Objetivo

### (4.1) Prioridad horaria de los docentes

$$ \min: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [123]:
def obj_prioridades(y_dict):

    OBJ1 = gp.LinExpr()

    count = 0
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * y_dict[p.id, b.id].variable
            count += 1

    print("obj_prioridades terms:", count)

    return OBJ1


### (4.2) Minimizar/Maximizar dias con clase por profesor
$$ \min: \sum_{p,d} {\alpha_p} {z_{pd}}$$
donde $\alpha_p = 1$ si se minimiza, $\alpha_p = -1$ si se maximiza y $\alpha_p = 0$ si es irrelevante

In [124]:
def obj_min_max_dias(z_dict):

    OBJ2 = gp.LinExpr()
    DP = {}

    count = 0
    for p in profesores:
        
        D_p = gp.quicksum(z_dict[p.id,d].variable for d in dias_ids)
        DP[p.id] = D_p

        # filtrar si el profesor prefiere minimizar o maximizar o ninguno
        match p.min_max_dias:
            case "min":
                OBJ2 += D_p
                count += len(dias_ids)
            case "max":
                OBJ2 -= D_p
                count += len(dias_ids)
            case None:
                pass

    print("obj_min_max_dias terms:", count)

    return OBJ2, DP



### (4.3) Minimizar horas puente por grupo (se incluye directamente como restriccion)

$$ \min: -\sum_{g,d,h} {x_{gd(h)}·x_{gd(h+1)}} $$

In [125]:
def obj_horas_puente_grupos(x_dict):
    OBJ3 = gp.QuadExpr()

    count = 0
    for g in grupos:
        for d in dias_ids:
            OBJ3 += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
            count += 1

    print("obj_horas_puente_grupos terms:", count)

    return OBJ3


### (4.4) Minimizar superposicion de electivas

$$ \min:  \sum_{b} \sum_{m, m'}{u_{mb} \times SE_{m,m'} \times u_{m'b}} = 0 $$

para todo par (m, m') de electivas

In [126]:
def obj_superposicion_electivas(u_dict):
    OBJ4 = gp.QuadExpr()

    count = 0
    for m1 in electivas(materias):
        for m2 in electivas(materias):
            for b in bloques_horario_ids:
                OBJ4 += u_dict[m1.id, b].variable * superposicion_electivas[m1.id, m2.id].value * u_dict[m2.id, b].variable
                count += 1

    print("obj_superposicion_electivas terms:", count)

    return OBJ4

### (4.5) Evitar horas excepcionales

$$ \min: \sum_{m} \sum_{b \in \mathcal{E}_m} {u_{mb}} $$

In [127]:
def obj_horarios_excepcionales(u_dict, prac=False):

    objetivo = gp.LinExpr()

    count = 0
    for m in materias:

        if (not prac and m.teo_prac == "prac") or (prac and m.teo_prac != "prac"):
            continue

        for b in bloques_horario.values():
            excep = b.horario.turnos_excepcional
            if len(intersection(excep, m.turnos()))>0:
                objetivo += u_dict[m.id, b.id].variable
                count += 1
        
    print("obj_horarios_excepcionales terms:", count)

    return objetivo


Guardar instancia en formato json

In [128]:
# # recargar input original

# import copy

# anios = copy.deepcopy(ANIOS)
# dias = copy.deepcopy(DIAS)
# horarios = copy.deepcopy(HORARIOS)
# bloques_horario = copy.deepcopy(BLOQUES_HORARIO)
# profesores = copy.deepcopy(PROFESORES)
# grupos = copy.deepcopy(GRUPOS)
# materias = copy.deepcopy(MATERIAS)
# superposicion = copy.deepcopy(SUPERPOSICION)
# superposicion_electivas = copy.deepcopy(SUPERPOSICION_ELECTIVAS)

In [129]:
# #filtrar por grupos

# grupos = [g for g in grupos 
#                        if g.anio in [
#                         #    1,
#                         #    2,
#                         #    3,
#                            4,
#                            5,
#                        ]
#                     #    and g.carrera in [
#                     #     #    "CIV",
#                     #     #    "IND",
#                     #     #    "TEL",
#                     #     #    "INF",
#                     #     #    "DIA",
#                     #        "LINF",
#                     #     #    None
#                     #    ]
#                     #    and g.particion != 1
#                         # and not g.recurse
#                     ]

# # grupos: list[Grupo] = [g for g in grupos if not (g.anio == 3 and g.carrera == "INF") and not (g.anio == 4 and g.carrera == "LINF")]

# # materias = [m for m in materias if m.nombre not in ["Cam"]]

# ms = []
# for m in materias:
#     gs_m = []
#     for g in grupos:
#         if g in m.grupos:
#             gs_m.append(g)

#     if len(gs_m) > 0:
#         m.grupos = gs_m
#         ms.append(m)

# materias: list[Materia] = ms


# # ajustar conjunto de profesores y grupos segun las materias filtradas

# ps = []
# gs = []

# for m in materias:
#     # print(str(m), ":", [str(g) for g in m.grupos], [str(p) for p in m.profesores], m.carga_horaria, m.cantidad_dias)
#     for p in m.profesores:
#         if p not in ps:
#             ps.append(p)
#     for g in m.grupos:
#         if g not in gs:
#             gs.append(g)

# for p in ps: # ajustar materias que puede dictar un profesor
#     p.lista_materias = [m for m in p.lista_materias if m['nombre_materia'] in [str(m) for m in materias]]
#     # print(f"{p}: {set([str(m) for m in materias if p in m.profesores])} =? {[f'{m['nombre_materia'], m['grupos_max']}' for m in p.lista_materias]}")

# grupos: list[Grupo] = gs
# grupos.sort(key=lambda x: x.id)

# profesores: list[Profesor] = ps

# anios = [a for a in anios if a in [g.anio for g in grupos]]

# print([str(g) for g in grupos])

In [130]:
# generate_instance_json("../instances/instance_4th&5th_2026sem1.json", materias, grupos, profesores, dias, horarios, turnos, superposicion, superposicion_electivas, num_salones)

# Resolver

## Compilar modelo

In [131]:
# Create a new model
model: gp.Model = gp.Model("timetable")

In [132]:
# variables
u_dict, v_dict, w_dict, x_dict, y_dict, z_dict = initialize_variables(materias, bloques_horario, dias, profesores, grupos)
create_variables(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)

u:  13200
v:  825
w:  19305
x:  2320
y:  9360
z:  585


### Seleccionar restricciones

In [133]:
constr_superposicion(model, u_dict)

In [134]:
constr_carga_horaria(model, u_dict)

In [135]:
constr_dias_materia(model, v_dict)

In [136]:
constr_max_min_horas(model, u_dict, v_dict)

In [137]:
constr_turnos_materia(model, u_dict)


In [138]:
constr_horas_consecutivas(model, u_dict, v_dict)


In [139]:
constr_dias_consecutivos(model, v_dict)
# constr_horarios_excepcionales(model, u_dict, 7, 1)   # (opcional) limitar cantidad de horas excepcionales

In [140]:
constr_no_disponible_profesor(model, u_dict, w_dict)


In [141]:
constr_unica_materia_profesor(model, u_dict, w_dict)


In [142]:
constr_limitar_profesores_materia(model, w_dict)


In [143]:
constr_cantidad_profesores(model, w_dict)


In [144]:
constr_grupos_max_profesor(model, w_dict)

In [145]:
constr_definir_y(model, y_dict, u_dict, w_dict)


In [146]:
constr_definir_z(model, z_dict, y_dict)

In [147]:
constr_cantidad_salones(model, u_dict, num_salones)


In [148]:
constr_teo_prac(model, u_dict)

In [149]:
constr_definir_x(model, x_dict, u_dict)


In [150]:
constr_horas_puente_grupos(model, x_dict)

### Funcion objetivo 

In [151]:
# funcion objetivo

objectives = { # name : (expr, weight)
    
    "prioridades" : (obj_prioridades(y_dict), 0), # weight = 1
    "min_max_dias" : (obj_min_max_dias(z_dict)[0], 0), # weight = 5
    "superposicion_electivas" : (obj_superposicion_electivas(u_dict), 0), # weight = 10
    "horarios_excepcionales (practico)" : (obj_horarios_excepcionales(u_dict, prac=True), 1), # weight = 50
    "horarios_excepcionales" : (obj_horarios_excepcionales(u_dict, prac=False), 2), # weight = 100

}


obj_prioridades terms: 4045
obj_min_max_dias terms: 175
obj_superposicion_electivas terms: 8000
obj_horarios_excepcionales terms: 110
obj_horarios_excepcionales terms: 865


In [152]:

OBJ = gp.QuadExpr()
for name in objectives:
    expr, weight = objectives[name]
    OBJ += weight * expr

model.setObjective(OBJ, GRB.MINIMIZE)


In [153]:
def set_starting_solution(solution_path):

    print("Setting starting solution...")

    data = read_json_solution(solution_path)

    for mb_pair, var in u_dict.items():
        var.variable.Start = 1 if mb_pair in data["u_dict"] else 0
            
    for md_pair, var in v_dict.items():
        var.variable.Start = 1 if md_pair in data["v_dict"] else 0

    for mp_pair, var in w_dict.items():
        var.variable.Start = 1 if mp_pair in data["w_dict"] else 0

## Optimizar

In [ ]:
model.setParam("TimeLimit", 2*60*60)
# model.setParam('OutputFlag', 1)
model.Params.MIPGap = 0.01/100
# model.Params.PoolSolutions = 10 # Number of solutions to find
# model.Params.PoolSearchMode = 0 # 0, 1, 2

model.optimize()

# nSolutions = model.SolCount
# print(f"Number of solutions found: {nSolutions}")

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 131502 rows, 68919 columns and 306591 nonzeros
Model fingerprint: 0x11cd310f
Model has 41176 quadratic constraints
Model has 12309 general constraints
Variable types: 0 continuous, 68919 integer (45595 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 2e+00]
Presolve removed 115619 rows and 55857 columns
Presolve time: 0.34s
Presolved: 42741 rows, 20413 columns, 124105 nonzeros
Presolved model has 29 SOS constraint(s)
Variable types: 0 continuous, 20413 integer (19670 binary

In [154]:
set_starting_solution("../results/solution_test.json")


Setting starting solution...


In [155]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 131502 rows, 68919 columns and 306591 nonzeros
Model fingerprint: 0x98c9592d
Model has 41176 quadratic constraints
Model has 12309 general constraints
Variable types: 0 continuous, 68919 integer (45595 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 2e+00]

User MIP start produced solution with objective 15 (0.15s)
Loaded user MIP start with objective 15

Presolve removed 115619 rows and 55857 columns
Presolve time: 0.29s
Presolved: 42741 rows, 20413 columns, 124105 nonzeros
Presolved model has 29 SOS constrai

### Analizar no-factibilidad 

In [678]:
# if model.Status == GRB.INFEASIBLE:
#     model.computeIIS()
#     model.write('iismodel.ilp')

#     # Print out the IIS constraints and variables
#     print('\nThe following constraints and variables are in the IIS:')
#     for c in model.getConstrs():
#         if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

#     for v in model.getVars():
#         if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
#         if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

## Imprimir horarios

In [ ]:
# save_solution_json(u_dict, v_dict, w_dict, "../results/solution_test.json")

In [ ]:

if not model.Status == GRB.INFEASIBLE:

    print('Obj: %g' % model.ObjVal)
    for name in objectives:
        expr, weight = objectives[name]
        print(f"\t{name}: \t{" "*(35-len(name))}{round(expr.getValue())} \t(peso={weight})")
        # print('=============================================================')
            
    print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)

Obj: 15
	prioridades: 	                        954 	(peso=0)
	min_max_dias: 	                       78 	(peso=0)
	superposicion_electivas: 	            50 	(peso=0)
	horarios_excepcionales (practico): 	  1 	(peso=1)
	horarios_excepcionales: 	             7 	(peso=2)

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  10
			1		2		3		4		5
··········································································································
0_start-0_end		---		81		---		---		16
1_start-1_end		---		81		57		85		16
2_start-2_end		---		81		57		85		89
3_start-3_end		---		88		57		88		89
4_start-4_end		0		88		56		88		81
5_start-5_end		0		---		56		88		81
6_start-6_end *		---		---		56		---		---

 Grupo:  11
			1		2		3		4		5
··········································································································
0_start-0_end		81		56		---		---		89
1_start-1_end		81		56		---		57		89
2_start-2_end		0		56		---		5

In [330]:
if not model.Status == GRB.INFEASIBLE:
    print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)


 Profesor:  3 3
	1	2	3	4	5
0_start-0_end	---	---	41	---	---
1_start-1_end	---	---	41	---	---
2_start-2_end	---	---	41	---	---
9_start-9_end	60	60	---	---	---
10_start-10_end	60	60	---	---	---

 Profesor:  4 4
	1	2	3	4	5
1_start-1_end	---	0	---	---	---
2_start-2_end	0	0	---	---	---
3_start-3_end	0	---	---	---	---
4_start-4_end	0	---	---	---	---
5_start-5_end	0	---	---	---	---
7_start-7_end	---	---	---	---	0
8_start-8_end	---	---	---	---	0

 Profesor:  5 5
	1	2	3	4	5
9_start-9_end	---	---	---	---	91
10_start-10_end	---	---	91	---	---
11_start-11_end	---	---	91	---	---
12_start-12_end	---	---	---	91	---
13_start-13_end	---	---	---	91	---
15_start-15_end	91	---	---	---	---

 Profesor:  6 6
	1	2	3	4	5
12_start-12_end	---	69	---	---	---
13_start-13_end	---	69	---	---	---
14_start-14_end	---	69	---	---	---

 Profesor:  7 7
	1	2	3	4	5
10_start-10_end	---	---	---	---	31
11_start-11_end	---	---	---	---	31
12_start-12_end	---	---	---	31	---
13_start-13_end	---	---	---	31	---

 Profesor:  9 9
	1	

## Estadísticas de resultados

### Promedios de prioridad

In [331]:

if not model.Status == GRB.INFEASIBLE and "prioridades" in objectives:
    
    CARGA_HORARIA_TOTAL = np.sum([round(y_dict[p, b].variable.X)
                              for p in profesores_ids
                              for b in bloques_horario_ids
                              ])

    obj = objectives["prioridades"][0].getValue()
    print(obj)
    print("Promedio general de prioridad horaria: ", obj/CARGA_HORARIA_TOTAL)
          

# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


954.0
Promedio general de prioridad horaria:  1.394736842105263


In [332]:
# promedio por profesor:
"""
This script calculates and prints the average workload per professor based on their assigned priorities and time blocks.
Variables:
    profesores (list): List of professor objects, each containing their priorities.
    y_dict (dict): Dictionary mapping professor IDs and time block IDs to Gurobi variables.
    bloques_horario_ids (list): List of time block IDs.
For each professor in the list `profesores`, the script performs the following steps:
1. Initializes a quadratic expression `OBJ_p` to accumulate the weighted priorities.
2. Iterates over the professor's priorities to update `OBJ_p` based on the priority value and corresponding Gurobi variable.
3. Calculates the total workload `CARGA_HORARIA_p` for the professor by summing the relevant Gurobi variables.
4. If the total workload is non-zero, it prints the professor's name, total workload, and the rounded average priority value per workload unit.
5. If the total workload is zero, it prints the professor's name and indicates that they have no workload.

"""
if not model.Status == GRB.INFEASIBLE:
    for p in profesores:

        OBJ_p = gp.QuadExpr()
        # for m in materias:
        #     for pr in p.prioridades:
        #             b = pr.bloque_horario
        #             A = pr.value
        #             OBJ_p += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id].variable

        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ_p += A * y_dict[p.id, b.id].variable
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()
        CARGA_HORARIA_p = round(gp.quicksum(y_dict[p.id, b].variable for b in bloques_horario_ids).getValue())

        if CARGA_HORARIA_p != 0:
            print(str(p), "\t", CARGA_HORARIA_p, "\t", round(round(OBJ_p.getValue())/CARGA_HORARIA_p,3))
        else:
            print(str(p), "sin carga horaria")

    


3 	 7 	 1.571
4 	 8 	 1.0
5 	 6 	 1.333
6 	 3 	 1.0
7 	 4 	 2.0
9 	 4 	 1.25
11 	 5 	 1.8
12 	 5 	 1.4
13 	 8 	 1.5
14 	 12 	 1.0
15 	 12 	 1.083
16 	 3 	 1.0
17 	 3 	 1.0
18 	 6 	 1.333
21 	 4 	 3.0
26 	 10 	 1.0
28 	 4 	 1.0
29 	 4 	 1.5
30 	 8 	 1.25
31 	 5 	 2.0
32 	 6 	 1.0
33 	 4 	 1.75
34 	 4 	 1.25
36 	 15 	 1.6
37 	 4 	 3.0
38 	 5 	 2.6
39 	 8 	 1.0
42 	 4 	 1.0
43 	 6 	 1.833
44 	 3 	 1.0
47 	 7 	 2.143
48 	 5 	 1.4
49 	 4 	 1.25
50 	 7 	 1.571
53 	 5 	 1.0
54 	 4 	 2.5
55 	 5 	 1.0
57 	 4 	 1.0
58 	 4 	 2.5
59 	 12 	 1.583
60 	 4 	 1.0
61 	 11 	 1.818
62 	 5 	 2.2
64 	 5 	 1.2
66 	 6 	 1.0
68 	 4 	 1.0
71 	 10 	 2.0
72 	 4 	 1.5
74 	 10 	 1.0
75 	 12 	 1.0
76 	 7 	 1.0
77 	 5 	 1.4
80 	 4 	 2.25
82 	 7 	 1.286
84 	 4 	 2.0
85 	 10 	 1.0
86 	 4 	 2.0
87 	 2 	 1.0
88 	 3 	 1.0
91 	 14 	 1.643
92 	 5 	 1.0
94 	 3 	 2.0
95 	 5 	 1.4
97 	 4 	 1.25
99 	 2 	 1.5
100 	 6 	 1.5
101 	 2 	 3.0
102 	 9 	 1.0
103 	 2 	 2.0
104 	 6 	 1.0
106 	 5 	 2.8
107 	 5 	 2.2
108 	 4 	 1.0
109 	 5 	

In [333]:
# # DIAS CLASE POR PROFESOR

# print("DIAS CON CLASE")
# for p in profesores:
#     if p.min_max_dias is not None:
#         print(str(p), ": ", round(DP[p.id].getValue()), p.min_max_dias)

In [334]:
# HORAS PUENTE

# HP = gp.QuadExpr()
# for g in grupos_reales:
#     HPg = gp.QuadExpr()
#     for d in dias_ids:
#         HPg += gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
#         HPg += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
#         HPg += - 1
#     # print("horas puente grupo", str(g), ": ", HPg.getValue())
#     HP += HPg

# print("Total de horas puente: ", HP.getValue())

#### Distribucion de prioridades

In [335]:
# distribucion de prioridades
"""
This script calculates and prints the distribution of priorities and their averages for a given set of professors and their time block priorities.
Variables:
    promedios_prioridad (dict): A dictionary to store the average priority values for each priority level.
    valores_prioridad (dict): A dictionary to store the count of each priority level.
    profesores (list): A list of professor objects, each containing their priority information.
    y_dict (dict): A dictionary containing the decision variables for the optimization model.
Process:
1. Initialize dictionaries to store priority averages and counts.
2. Iterate over each professor and their priorities:
    - Skip if the priority value is 0.
    - Increment the count for the given priority value.
    - Update the average priority value based on the decision variable.
3. Print the count and percentage distribution of each priority level.
4. Print the average priority values and their percentage distribution.

"""

if not model.Status == GRB.INFEASIBLE:
    promedios_prioridad = {}
    valores_prioridad = {}
    for i in range(1,4):
        promedios_prioridad[i] = 0
        valores_prioridad[i] = 0

    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            if A == 0:
                continue
            valores_prioridad[A] += 1
            # for m in materias:
            #     promedios_prioridad[A] += round(w_dict[m.id, p.id].variable.X * u_dict[m.id, b.id].variable.X)
            promedios_prioridad[A] += round(y_dict[p.id, b.id].variable.X)

    print(valores_prioridad)
    total = np.sum([valores_prioridad[i] for i in valores_prioridad])
    print(*[str(i) + ": " + str(round(valores_prioridad[i]/total*100))+"%" for i in valores_prioridad], sep=", ")

    print(promedios_prioridad)
    total = np.sum([promedios_prioridad[i] for i in promedios_prioridad])
    if total != 0:
        print(*[str(i) + ": " + str(round(promedios_prioridad[i]/total*100))+"%" for i in promedios_prioridad], sep=", ")
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()

        # if CARGA_HORARIA_p != 0:
        #     print(str(p), ": ", OBJ_p.getValue()/CARGA_HORARIA_p)
        # else:
        #     print(str(p), "sin carga horaria")

{1: 3179, 2: 442, 3: 424}
1: 79%, 2: 11%, 3: 10%
{1: 505, 2: 88, 3: 91}
1: 74%, 2: 13%, 3: 13%


### Ocupacion de salones

In [336]:
# nivel de ocupacion
if not model.Status == GRB.INFEASIBLE:
    niveles = {}

    for b_id in bloques_horario:
        salones_ocupados = int(gp.quicksum(u_dict[m, b_id].variable for m in materias_ids).getValue())
        nivel = int(salones_ocupados/num_salones*100)
        # print(str(bloques_horario[b_id]), ": ", salones_ocupados,",", str(nivel) + "%")
        niveles[b_id] = nivel

    print_timetable_salones(dias, horarios, niveles)

		1	2	3	4	5
··········································································································
0_start-0_end	61%	61%	46%	53%	92%
1_start-1_end	76%	84%	69%	92%	92%
2_start-2_end	69%	76%	76%	92%	76%
3_start-3_end	69%	76%	69%	76%	76%
4_start-4_end	69%	92%	61%	76%	69%
5_start-5_end	69%	76%	46%	61%	69%
6_start-6_end	0%	0%	7%	7%	7%
7_start-7_end	15%	30%	15%	23%	15%
8_start-8_end	15%	30%	15%	30%	23%
9_start-9_end	84%	61%	38%	69%	53%
10_start-10_end	76%	76%	46%	84%	92%
11_start-11_end	84%	69%	69%	69%	69%
12_start-12_end	69%	92%	92%	92%	69%
13_start-13_end	76%	92%	76%	100%	69%
14_start-14_end	76%	61%	53%	76%	69%
15_start-15_end	23%	23%	15%	38%	23%
